# Account Portfolio & Revenue Decision Support

This notebook analyses an account portfolio to support prioritisation, revenue risk management, and resource allocation decisions in an account management context.
It focuses on portfolio concentration, segment behaviour, and early signals of account risk.

## Business question

Which accounts drive sustainable value, which are at risk, and how should account management effort be prioritised?

## Data

This notebook uses `account_portfolio_data.csv` located in the `data/` folder.


## Approach

- Assess revenue concentration to understand portfolio risk.
- Segment accounts into actionable groups (High value / Core value / Long tail).
- Identify high-value accounts showing risk signals.
- Produce a stakeholder-facing snapshot for prioritisation conversations.

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("../data/account_portfolio_data.csv")
df.head()

In [ ]:
# Portfolio summary
total_rev = df["annual_revenue"].sum()
total_accounts = df.shape[0]
(total_accounts, total_rev)

In [ ]:
# Revenue concentration
df_sorted = df.sort_values("annual_revenue", ascending=False).copy()
df_sorted["cum_rev_share"] = df_sorted["annual_revenue"].cumsum() / total_rev
df_sorted["account_rank"] = np.arange(1, len(df_sorted)+1)

# What share of revenue comes from top X accounts?
for x in [5,10,20,30]:
    share = df_sorted.head(x)["annual_revenue"].sum() / total_rev
    print(f"Top {x} accounts revenue share: {share:.1%}")

In [ ]:
# Segment performance and risk view
seg = df.groupby("segment", as_index=False).agg(
    accounts=("account","count"),
    revenue=("annual_revenue","sum"),
    avg_engagement=("engagement_score","mean"),
    risk_rate=("risk_flag","mean")
)
seg["revenue_share"] = seg["revenue"] / total_rev
seg.sort_values("revenue", ascending=False)

In [ ]:
# Identify high-value at-risk accounts (high revenue + risk flag)
threshold = df["annual_revenue"].quantile(0.85)
hv_risk = df[(df["annual_revenue"] >= threshold) & (df["risk_flag"])].sort_values("annual_revenue", ascending=False)
hv_risk.head(15)

In [ ]:
import matplotlib.pyplot as plt

# Visual: Top 10 accounts by revenue (stakeholder snapshot)
top10 = df.sort_values("annual_revenue", ascending=False).head(10)

plt.figure(figsize=(9,5))
plt.bar(top10["account"], top10["annual_revenue"])
plt.xticks(rotation=60, ha="right")
plt.ylabel("Annual revenue")
plt.title("Top 10 Accounts by Annual Revenue")
plt.tight_layout()

plt.savefig("../visuals/insight_snapshot.png", dpi=200)
plt.show()

In [ ]:
# Key findings table (ready to reference in README)
top_accounts = df.sort_values("annual_revenue", ascending=False).head(15)[
    ["account","annual_revenue","segment","engagement_score","support_tickets_qtr","nps","risk_flag"]
]
top_accounts